# KBMOD Demo

In [ ]:
import os
import numpy as np

from pathlib import Path

from kbmod.run_search import *
from kbmod.search import *

# Setup file paths

There are at least two file paths you need to setup in order to run kbmod:
1. The im_filepath provides a path to the input images.
1. The res_filepath provides a path to the directory where the output results will be stored.

A time and psf file can optionally be specified.

If you already have data files, you can use those. Below we use the data in `data/demo`. You can also create your own fake data using `fake_data_creator.py`.

In [ ]:
# Define the path for the data.
im_filepath = "../data/demo"
print(os.listdir(im_filepath))

In [ ]:
res_filepath = "./fake_results"
if not Path(res_filepath).is_dir():
    print(f"Directory {res_filepath} does not exist. Creating.")
    os.mkdir(res_filepath)

# Run KBMOD

The standard approach to running KBMOD is to perform a grid search over all starting pixels and a grid of velocities. The velocities are defined by steps in velocity space (in pixels per day) and angles. Let’s do a grid search with:
* 21 different velocity steps from 0 pixels per day and 20 pixels per day
* 11 different angles from 0.5 below the default angle (computed based on the ecliptic) to 0.5 above.

We require objects to be found in at least 7 of the time steps.

In [ ]:
results_suffix = "DEMO"

# The demo data has an object moving at x_v=10 px/day
# and y_v = 0 px/day. So we search velocities [0, 20].
v_min = 0
v_max = 20
v_steps = 21
v_arr = [v_min, v_max, v_steps]

# and angles [-0.5, 0.5]
ang_below = 0.5
ang_above = 0.5
ang_steps = 11
ang_arr = [ang_below, ang_above, ang_steps]

# There are 10 images in the demo data. Make sure we see
# the object in at least 7.
num_obs = 7

input_parameters = {
    # Required
    "im_filepath": im_filepath,
    "res_filepath": res_filepath,
    "time_file": None,
    "output_suffix": results_suffix,
    "v_arr": v_arr,
    "ang_arr": ang_arr,
    # Important
    "num_obs": num_obs,
    "do_mask": True,
    "lh_level": 10.0,
    "gpu_filter": True,
    # Fine tuning
    "sigmaG_lims": [15, 60],
    "mom_lims": [37.5, 37.5, 1.5, 1.0, 1.0],
    "peak_offset": [3.0, 3.0],
    "chunk_size": 1000000,
    "stamp_type": "cpp_median",
    "eps": 0.03,
    "clip_negative": True,
    "mask_num_images": 10,
    "cluster_type": "position",
    # Override the ecliptic angle for the demo data since we
    # know the true angle in pixel space.
    "average_angle": 0.0,
    "result_filename": "./fake_results/results.ecsv",
    "save_all_stamps": True,
}

rs = SearchRunner()
results = rs.run_search_from_config(input_parameters)

We then check that results were written to an output directory.

In [ ]:
if os.path.exists(res_filepath):
    files = os.listdir(res_filepath)
    print(files)

# Analyzing Results

The run function we used returns a `ResultList` containing the individual results of the run. We can perform basic actions on this data structure such as sorting it, extracting individual results, or performing additional filtering.

In [ ]:
print(f"Search found {results.num_results()} results.")

# Sorting one result doesn't do anything.
results.sort()

# Output the trajectory.
best_result = results.results[0]
print(best_result.trajectory)

We can also plot the different curves for the result.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1)
axs[0].plot(results.all_times, best_result.psi_curve)
axs[0].set_title("Psi")

axs[1].plot(results.all_times, best_result.phi_curve)
axs[1].set_title("Psi")

axs[2].plot(results.all_times, best_result.likelihood_curve)
axs[2].set_title("Likelihood")

For additional analysis steps, please refer to the kbmod_analysis_demo notebook which uses the data generated by this notebook.